In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE142153"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE142153"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE142153.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE142153.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE142153.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Human PBMCs: Healthy vs Diabetic nephropathy vs ESRD"
!Series_summary	"Transcriptional profiling of human PBMCs comparing healthy controls, patients with diabetic nephropathy and patients with ESRD. PBMCs were analyzed as they mediate inflammatory injury. Goal was to determine effects of increasing severity of diabetic nephropathy on global PBMC gene expression. Microarray analysis of PBMCs taken from patients with varying degrees of diabetic nephropathy."
!Series_overall_design	"3 condition experiment - Healthy control (10) vs diabetic nephropathy (23) vs ESRD (7)"
Sample Characteristics Dictionary:
{0: ['tissue: peripheral blood'], 1: ['diagnosis: healthy control', 'diagnosis: diabetic nephropathy', 'diagnosis: ESRD']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptional profiling data
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (diabetic nephropathy/chronic kidney disease):
# Row 1 contains diagnosis information with multiple values
trait_row = 1

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

# Convert trait values to binary (0 for control, 1 for disease)
def convert_trait(value):
    if isinstance(value, str):
        # Extract the value after colon if exists
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Convert based on diagnosis
        if 'healthy control' in value.lower():
            return 0  # Control
        elif 'diabetic nephropathy' in value.lower() or 'esrd' in value.lower():
            return 1  # Disease (Diabetic nephropathy or ESRD both indicate kidney disease)
    return None

# Since age data is not available, we define a placeholder function
def convert_age(value):
    return None

# Since gender data is not available, we define a placeholder function
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical features dataframe
    print("Preview of clinical features:")
    print(preview_df(clinical_features_df))
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to a CSV file
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM4221568': [0.0], 'GSM4221569': [0.0], 'GSM4221570': [0.0], 'GSM4221571': [0.0], 'GSM4221572': [0.0], 'GSM4221573': [0.0], 'GSM4221574': [0.0], 'GSM4221575': [0.0], 'GSM4221576': [0.0], 'GSM4221577': [0.0], 'GSM4221578': [1.0], 'GSM4221579': [1.0], 'GSM4221580': [1.0], 'GSM4221581': [1.0], 'GSM4221582': [1.0], 'GSM4221583': [1.0], 'GSM4221584': [1.0], 'GSM4221585': [1.0], 'GSM4221586': [1.0], 'GSM4221587': [1.0], 'GSM4221588': [1.0], 'GSM4221589': [1.0], 'GSM4221590': [1.0], 'GSM4221591': [1.0], 'GSM4221592': [1.0], 'GSM4221593': [1.0], 'GSM4221594': [1.0], 'GSM4221595': [1.0], 'GSM4221596': [1.0], 'GSM4221597': [1.0], 'GSM4221598': [1.0], 'GSM4221599': [1.0], 'GSM4221600': [1.0], 'GSM4221601': [1.0], 'GSM4221602': [1.0], 'GSM4221603': [1.0], 'GSM4221604': [1.0], 'GSM4221605': [1.0], 'GSM4221606': [1.0], 'GSM4221607': [1.0]}
Clinical features saved to ../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE142153.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Assume gene data is available

# Extract gene data
try:
    # Extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # Print the first 20 gene/probe identifiers
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20].tolist())
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print(f"File path: {matrix_file}")
    print("Please check if the file exists and contains the expected markers.")


SOFT file: ../../input/GEO/Chronic_kidney_disease/GSE142153/GSE142153_family.soft.gz
Matrix file: ../../input/GEO/Chronic_kidney_disease/GSE142153/GSE142153_series_matrix.txt.gz
Gene data shape: (30811, 40)
First 20 gene/probe identifiers:
['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100111', 'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156', 'A_23_P100189', 'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100278', 'A_23_P100292', 'A_23_P100315', 'A_23_P100344', 'A_23_P100355', 'A_23_P100386']


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these are Agilent microarray probe IDs (starting with 'A_23_P'),
# not standard human gene symbols. These probe IDs will need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check if there are any columns that might contain gene information beyond what we've seen
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Look for additional columns that might contain gene symbols
# Since we only have 'ID' and 'ENTREZ_GENE_ID', check if we need to use Entrez IDs for mapping
gene_id_col = 'ID'
gene_symbol_col = None

# Check various potential column names for gene symbols
for col_name in ['GENE_SYMBOL', 'SYMBOL', 'GENE', 'GENE_NAME', 'GB_ACC']:
    if col_name in gene_annotation.columns:
        gene_symbol_col = col_name
        break

# If no dedicated symbol column is found, we'll need to use ENTREZ_GENE_ID
if gene_symbol_col is None and 'ENTREZ_GENE_ID' in gene_annotation.columns:
    gene_symbol_col = 'ENTREZ_GENE_ID'
    print("\nNo direct gene symbol column found. Will use Entrez Gene IDs for mapping.")

if gene_id_col in gene_annotation.columns and gene_symbol_col is not None:
    print(f"\nSample mappings from '{gene_id_col}' to '{gene_symbol_col}':")
    sample_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].head(10)
    print(sample_mappings)
    
    # Check for non-null mappings to confirm data quality
    non_null_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].dropna(subset=[gene_symbol_col])
    print(f"\nNumber of probes with gene ID mappings: {len(non_null_mappings)}")
    print(f"Sample of valid mappings:")
    print(non_null_mappings.head(5))
else:
    print("Required mapping columns not found in the annotation data. Will need to explore alternative mapping approaches.")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apopt

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# Based on the preview, 'ID' contains probe identifiers and 'GENE_SYMBOL' contains gene symbols
probe_id_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

print(f"Using '{probe_id_col}' as probe identifier column and '{gene_symbol_col}' as gene symbol column")

# 2. Create a gene mapping dataframe using the two columns
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows and columns of gene expression data:")
print(gene_data.iloc[:5, :5])

# Save gene data to CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using 'ID' as probe identifier column and 'GENE_SYMBOL' as gene symbol column
Gene mapping dataframe shape: (30936, 2)
Sample of gene mapping:
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN
Gene expression data shape after mapping: (18440, 40)
First few rows and columns of gene expression data:
          GSM4221568  GSM4221569  GSM4221570  GSM4221571  GSM4221572
Gene                                                                
A1BG        1.689609    0.116657    0.729824    1.233267   -0.753530
A1BG-AS1    1.735950    1.614540    1.115200    0.556956    0.662771
A1CF       -5.546720   -6.946890   -9.237430   -7.895080   -6.992950
A2LD1       1.611080    1.368170    0.811205    1.717300    1.131080
A2M        -7.113060   -6.750490   -6.552580   -6.926450   -6.712910


Gene expression data saved to ../../output/preprocess/Chronic_kidney_disease/gene_data/GSE142153.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data shape: {gene_data.shape}")
    
    # Apply normalization
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Normalized gene data shape: {gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data that was already processed and saved
try:
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data shape: {clinical_data.shape}")
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Link the clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_data, gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Linked data shape after handling missing values: {linked_data.shape}")
        
        # Check for bias in trait and demographic features
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        
        # Validate the data quality and save cohort info
        note = "Dataset contains gene expression data from peripheral blood of healthy controls and patients with diabetic nephropathy and ESRD."
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=is_biased,
            df=linked_data,
            note=note
        )
        
        # Save the linked data if it's usable
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            linked_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data not usable for the trait study - not saving final linked data.")
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Loaded gene data shape: (18440, 40)
Normalized gene data shape: (18202, 40)


Normalized gene data saved to ../../output/preprocess/Chronic_kidney_disease/gene_data/GSE142153.csv
Loaded clinical data shape: (1, 40)
Linked data shape: (40, 18203)


Linked data shape after handling missing values: (40, 18203)
For the feature 'Chronic_kidney_disease', the least common label is '0.0' with 10 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Chronic_kidney_disease' in this dataset is fine.



Linked data saved to ../../output/preprocess/Chronic_kidney_disease/GSE142153.csv
